<a href="https://colab.research.google.com/github/ravishankar75/group44_cv_assignment/blob/main/CV_assignment_1_group44.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Group 44

## Group Member Names:
1. SAKTHI R (2023aa05940)
2. ROBERTSEKAR R (2023aa05823)
3. RAVISHANKAR R (2023aa05171)
4. KRISHNAKUMAR C (2023aa05273)

Dataset Link: https://drive.google.com/file/d/18ivVD85YKQqPH0Qhe2Ou10hjuPl-vWxA/view?usp=sharing


Choose any 1 dataset of your choice to perform the assignment.

# 1. Import the required libraries -- Score: 0.5 Marks

In [ ]:
import os
import zipfile
from google.colab import drive

import numpy as np
import pandas as pd

import cv2
from skimage.feature import local_binary_pattern #opencv2 does not expose LBP

# Use Sklearn SVM or Xgboost to train the model
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Use sklearn label encoder to convert to numeric value the label column
from sklearn.preprocessing import LabelEncoder

# Using Sklearn, split the dataset into training and test in 80:20 ratio
from sklearn.model_selection import train_test_split


# 2. Data Acquisition & Preparation -- Score: 1.5 Marks

For the problem identified by you, students have to find the data source themselves from any data source.

## 2.1 Data Acquisition -- Score: 0.5 Mark

Code for converting the above downloaded data into a form suitable for DL


In [7]:
##---------Type the code below this line------------------##

# Shared dataset link not working with gdown library due to permissions
# The images have been downloaded and uploaded to Google drive(My).

# The below steps necessary as on reconnect, google collab deletes the content folder

colab_folder = "/content"  # Default working directory in Google Colab
zip_file_name = "/content/drive/MyDrive/scene_classification.zip"


# Decompress the zip file, using with instead of try/finally
# check if data folder exists, if it exists then the dataset has been extracted

if not os.path.exists(os.path.join(colab_folder, "data")):

  print("Mounting Google Drive...")
  drive.mount('/content/drive') # Approve access for google desktop app

  print("Decompressing the zip file...")
  with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
      extract_path = os.path.join(colab_folder, "data")
      zip_ref.extractall(extract_path)

print( f"Decompression complete., Files extracted to {extract_path}")

# find the number of images for each category

for category in os.listdir(os.path.join(colab_folder, "data", "subset")):
  category_path = os.path.join(colab_folder, "data", "subset", category)
  if not os.path.isdir(category_path):
    continue
  num_images = len(os.listdir(category_path))
  print(f"Category: {category}, Number of Images: {num_images}")

# Define constants
NUM_IMAGES_PER_FOLDER = 500
DATASET = []

# Features to extract from images - Color, Shape and Texture

# Function to resize Image, convert all images to standard size
def resize_image(image):
    IMAGE_SIZE = (128, 128)  # Resize to 128x128
    resized_image = cv2.resize(image, IMAGE_SIZE)
    return resized_image

# Function to extract color details
# Forests are mostly green, Sea blue, Glaciers White etc
def extract_color_details(image):
   # OpenCv2 loads in BGR format, calculate histogram of intensity
   # Mask = None, All 3 channels, 8 Bins, Range of Intensity (0 to L-1) L=256
   hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
   hist = cv2.normalize(hist, hist).flatten()
   return hist

# Function to extract texture
# Texture refer to spatial arrangement of intensity or color patterns in an image
# Helps identify if there are irregular patterns (Mountain), uniform patterns (Sea, glacier), repeating patterns (Building, Street) etc
# Methods - Local Binary Factor, Gibor Filter, GLCM not incl- computational cost

def extract_texture_LBP(gray_image):
   # Input image has to be  grayscale
   # Compute the LBP of the image, using skimage.feature.localBinaryPattern
   # P=8, R=3.0
   lbp = local_binary_pattern(gray_image, 8, 3.0, method="uniform")

   return lbp.flatten()

def extract_texture_gabor(gray_image, psi):
  # Input image has to be  grayscale
  # Apply Gabor kernel, from the resultant image pixels, calculate mean & stddev

   gabor_kernel = cv2.getGaborKernel(ksize=(5, 5), sigma=5.0, theta=0.0, lambd=10.0, gamma=0.5, psi=psi)

   filtered_image = cv2.filter2D(gray_image, cv2.CV_8UC3, gabor_kernel)

   mean = np.mean(filtered_image)
   std_dev = np.std(filtered_image)
   return mean.flatten()

# Function to extract shape features
# Determine edges, Buildings/Street should have more edges than Sea/Glacier
# Use measure = no of edge pixels / total pixels in image

def extract_edge_features(gray_image):

  # detect edges using Canny algo
  edge_image = cv2.Canny(gray_image, threshold1=50, threshold2=150)
  no_of_edge_pixel = np.sum(edge_image > 0 ) # Any non black pixel

  return no_of_edge_pixel/gray_image.size

# Path to the base folder containing category subfolders
base_folder = os.path.join(colab_folder, "data", "subset")

# Iterate through category folders
for category in os.listdir(base_folder):
    category_path = os.path.join(base_folder, category)
    if not os.path.isdir(category_path):
        continue

    # Process up to NUM_IMAGES_PER_FOLDER images in this category
    image_count = 0
    for image_file in os.listdir(category_path):
        if image_count >= NUM_IMAGES_PER_FOLDER:
            break

        # Full path to image file
        image_path = os.path.join(category_path, image_file)
        if not os.path.isfile(image_path):
            continue

        # Process the image
        image = cv2.imread(image_path)

        if image is None:
            continue
        # Convert image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        dominant_colors = extract_color_details(image)
        texture_lbp = extract_texture_LBP(gray_image)
        texture_gabor_0 = extract_texture_gabor(gray_image, 0)
        texture_gabor_45 = extract_texture_gabor(gray_image, 45)
        texture_gabor_90 = extract_texture_gabor(gray_image, 90)

        edge_density = extract_edge_features(gray_image)


        # Append data to dataset
        DATASET.append({
            "img_name": image_path,
            "label": category,
            "dominant_colors": dominant_colors,
            "texture_lbp": texture_lbp,
            "texture_gabor_0": texture_gabor_0,
            "texture_gabor_45": texture_gabor_45,
            "texture_gabor_90": texture_gabor_90,
            "edge_density": edge_density
        })
        image_count += 1

# Convert dataset to a DataFrame
df = pd.DataFrame(DATASET)

# Save dataset to parquet
df.to_parquet('image_dataset.parquet', compression='snappy')


print("Dataset created and saved to 'image_dataset'")

df2 = pd.read_parquet("image_dataset.parquet")
df2.head()

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'label' column in the training set
train_df['label_encoded'] = label_encoder.fit_transform(train_df['label'])


## Extract features and labels from the DataFrames
X_train = train_df[['dominant_colors', 'texture_lbp', 'texture_gabor_0', 'texture_gabor_45', 'texture_gabor_90', 'edge_density']]
y_train = train_df['label_encoded']
X_test = test_df[['dominant_colors', 'texture_lbp', 'texture_gabor_0', 'texture_gabor_45', 'texture_gabor_90', 'edge_density']]
y_test = label_encoder.transform(test_df['label'])  # Transform test labels using the same encoder

# Convert the feature columns to NumPy arrays and flatten them
def flatten_features(df):
    features = []
    for _, row in df.iterrows():
        feature_row = []
        for col in ['dominant_colors', 'texture_lbp', 'texture_gabor_0', 'texture_gabor_45', 'texture_gabor_90']:
            feature_row.extend(np.array(row[col]).flatten())  # Flatten each feature
        feature_row.append(row['edge_density'])
        features.append(feature_row)
    return np.array(features)

X_train = flatten_features(X_train)
X_test = flatten_features(X_test)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the model on training dataset
model = SVC()
model.fit(X_train, y_train)

# Predict the data for held out test data and generate accuracy metrics
test_predictions = model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)

# Plot the classification accuracy metrics
print(f"Test Accuracy: {test_accuracy}")
print(classification_report(y_test, test_predictions))


Decompression complete., Files extracted to /content/data
Category: Building, Number of Images: 501
Category: Glacier, Number of Images: 501
Category: Sea, Number of Images: 501
Category: Mountains, Number of Images: 501
Category: Streets, Number of Images: 501
Category: Forest, Number of Images: 2745
Dataset created and saved to 'image_dataset'


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2400,) + inhomogeneous part.

## 2.2 Write your observations from the above.

1. Size of the dataset
2. Plot the distribution of the categories of the target / label.



## 2.2 Data Preparation -- Score: 1.0 Marks

Perform the data preprocessing that is required for the data that you have downloaded.


This stage depends on the dataset that is used.

## 3.1 Split the data into training set and testing set

In [ ]:
import os
import zipfile
from google.colab import drive

import numpy as np
import pandas as pd

import cv2
from skimage.feature import local_binary_pattern #opencv2 does not expose LBP
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns

##---------Code begins below this line------------------##

# Shared dataset link not working with gdown library due to permissions
# The images have been downloaded and uploaded to Google drive(My).

# The below steps necessary as on reconnect, Google Colab deletes the content folder

colab_folder = "/content"  # Default working directory in Google Colab
zip_file_name = "/content/drive/MyDrive/scene_classification.zip"

# Decompress the zip file, using with instead of try/finally
# Check if data folder exists; if it exists, then the dataset has been extracted
if not os.path.exists(os.path.join(colab_folder, "data")):

    print("Mounting Google Drive...")
    drive.mount('/content/drive')  # Approve access for Google desktop app

    print("Decompressing the zip file...")
    with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
        extract_path = os.path.join(colab_folder, "data")
        zip_ref.extractall(extract_path)

print(f"Decompression complete. Files extracted to {extract_path}")

# Find the number of images for each category
for category in os.listdir(os.path.join(colab_folder, "data", "subset")):
    category_path = os.path.join(colab_folder, "data", "subset", category)
    if not os.path.isdir(category_path):
        continue
    num_images = len(os.listdir(category_path))
    print(f"Category: {category}, Number of Images: {num_images}")

# Define constants
NUM_IMAGES_PER_FOLDER = 500
DATASET = []

# Features to extract from images - Color, Shape, and Texture

# Function to resize Image, convert all images to standard size
def resize_image(image):
    IMAGE_SIZE = (128, 128)  # Resize to 128x128
    resized_image = cv2.resize(image, IMAGE_SIZE)
    return resized_image

# Function to extract color details
def extract_color_details(image):
    # OpenCV loads in BGR format, calculate histogram of intensity
    hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

# Function to extract texture using LBP
def extract_texture_LBP(gray_image):
    # Compute the LBP of the image, using skimage.feature.localBinaryPattern
    lbp = local_binary_pattern(gray_image, 8, 3.0, method="uniform")
    return lbp.flatten()

# Function to extract texture using Gabor filters
def extract_texture_gabor(gray_image, psi):
    gabor_kernel = cv2.getGaborKernel(ksize=(5, 5), sigma=5.0, theta=0.0, lambd=10.0, gamma=0.5, psi=psi)
    filtered_image = cv2.filter2D(gray_image, cv2.CV_8UC3, gabor_kernel)
    mean = np.mean(filtered_image)
    std_dev = np.std(filtered_image)
    return [mean, std_dev]

# Function to extract shape features
def extract_edge_features(gray_image):
    edge_image = cv2.Canny(gray_image, threshold1=50, threshold2=150)
    no_of_edge_pixel = np.sum(edge_image > 0)
    return no_of_edge_pixel / gray_image.size

# Path to the base folder containing category subfolders
base_folder = os.path.join(colab_folder, "data", "subset")

# Process dataset and extract features
for category in os.listdir(base_folder):
    category_path = os.path.join(base_folder, category)
    if not os.path.isdir(category_path):
        continue

    image_count = 0
    for image_file in os.listdir(category_path):
        if image_count >= NUM_IMAGES_PER_FOLDER:
            break

        image_path = os.path.join(category_path, image_file)
        if not os.path.isfile(image_path):
            continue

        image = cv2.imread(image_path)
        if image is None:
            continue

        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        dominant_colors = extract_color_details(image)
        texture_lbp = extract_texture_LBP(gray_image)
        texture_gabor_0 = extract_texture_gabor(gray_image, 0)
        edge_density = extract_edge_features(gray_image)

        DATASET.append({
            "img_name": image_path,
            "label": category,
            "dominant_colors": dominant_colors,
            "texture_lbp": texture_lbp,
            "texture_gabor_0": texture_gabor_0,
            "edge_density": edge_density
        })
        image_count += 1

# Convert dataset to a DataFrame
df = pd.DataFrame(DATASET)

# Save dataset to Parquet
df.to_parquet('image_dataset.parquet', compression='snappy')
print("Dataset created and saved to 'image_dataset'")

# Dataset Size
print(f"Size of the dataset: {len(df)} images")

# Plotting category distribution
plt.figure(figsize=(10, 6))
sns.countplot(y="label", data=df, order=df["label"].value_counts().index)
plt.title("Category Distribution")
plt.xlabel("Count")
plt.ylabel("Category")
plt.show()

# Data Preprocessing
def preprocess_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    equalized_image = cv2.equalizeHist(gray_image)
    return equalized_image

# Splitting the dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])

print(f"Training set size: {len(train_df)} images")
print(f"Testing set size: {len(test_df)} images")

# Feature extraction
def extract_features(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None

    gray_image = preprocess_image(image)

    # Extract features
    dominant_colors = extract_color_details(image)
    texture_lbp = extract_texture_LBP(gray_image)
    edge_density = [extract_edge_features(gray_image)]  # Ensure this is a list

    # Check feature lengths
    try:
        # Concatenate features ensuring consistent lengths
        return np.concatenate([dominant_colors, texture_lbp, edge_density])
    except ValueError as e:
        print(f"Error concatenating features for {image_path}: {e}")
        return None

# Calculate expected feature length
sample_image_path = os.path.join(base_folder, os.listdir(base_folder)[0], os.listdir(os.path.join(base_folder, os.listdir(base_folder)[0]))[0])
sample_image = cv2.imread(sample_image_path)
gray_sample_image = preprocess_image(sample_image)
expected_feature_length = len(extract_color_details(sample_image)) + len(extract_texture_LBP(gray_sample_image)) + 1

# Process training and testing sets
def process_data(df):
    features = []
    labels = []
    for _, row in df.iterrows():
        feature = extract_features(row["img_name"])
        if feature is not None and feature.shape[0] == expected_feature_length:  # Check expected length
            features.append(feature)
            labels.append(row["label"])
    return np.array(features), np.array(labels)

train_features, train_labels = process_data(train_df)
test_features, test_labels = process_data(test_df)

# Normalizing the features
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

# PCA for dimensionality reduction
if train_features.shape[1] > 500:
    pca = PCA(n_components=500)
    train_features = pca.fit_transform(train_features)
    test_features = pca.transform(test_features)
    print(f"Reduced feature vector size: {train_features.shape[1]}")

# Save features
np.save('train_features.npy', train_features)
np.save('train_labels.npy', train_labels)
np.save('test_features.npy', test_features)
np.save('test_labels.npy', test_labels)

print("Processed features saved for training and evaluation.")

## 3.2 Feature Engineering -- Score: 3.5 Marks

* Extract the features from the images and concatenate them to create a single for the every images.

* You can choose from the feature processing techniques taught in the class : Low-level Vision: Histogram and Histogram equalization, Gray-scale transformation, Image Smoothing, Connected components in images.
Mid-level Vision:  Edge Detection using Gradients, Sobel, Canny; Line detection using Hough transforms; Semantic information using RANSAC;Image region descriptor using SIFT; Use case: Pedestrian detection Using HoG and SIFT descriptors and SVM

* Create multiple sets of features and store it in seperate dataframes so that you can later use it for training and comparing the models.

* Normalize the DataFrame

* Note : If the feature size is getting too large such that it is not fitting into the RAM of Colab or your system then you can either use PCA or resize the image to smaller dimenssion for reducing the numer of features



In [ ]:
##---------Type the answer below this line------------------##

# 4. Model Building - Score: 2.0 Marks

## 4.1 Model Building - Score: 1.5 Marks
* Use any 1 classical machine learning algorithm such as : SVM , Xgboost etc. to train the model
* Train the model on different kinds of feature combination dataframe you created in 3.

In [ ]:
##---------Type the code below this line------------------##

## 4.2 Validation matrix - Score: 0.5 Marks

Print the model accuracy and F1 Score


In [ ]:
##---------Type the answer below this line------------------##

# 5. Model Inference & Evaluation - Score: 1 Mark

Plot any 5 random test images and their predicted and actual true labels using the model and feature set which gave you the best accuracy/F1 score.


In [ ]:
##---------Type the code below this line------------------##

Justify your choice/inution of feature selection based on the performance of model such that why a particualr set have features might have performed well.


In [ ]:
##---------Type the answers below this line------------------##

# 6. Documentation, Study presentation and Code Quality -- Score: 1.5 Marks

### NOTE


All Late Submissions will incur a <b>penalty of -2 marks </b>. So submit your assignments on time.

Good Luck